

### Generación de Modelos Predictivos con LOGISTIC REGRESSION

Este tipo de modelos requiere una revisión previa de colienalidad.  Tambien permiten obtener información del peso de las variables en el modelo.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold

# Cargar tu DataFrame, reemplaza 'tu_dataframe.csv' con el nombre de tu archivo CSV
dfbase = pd.read_pickle('checkpoints/DATOS_LIMPIOS.pkl')

Cargamos el archivo ya preparado de DATOS_LIMPIOS y separamos las variables independientes en **X** y la dependiente en **y**

In [100]:
df = dfbase.copy()

In [101]:
df = df[df['LABEL']!=-1]

In [102]:
X = df.drop('LABEL', axis=1)
y = df['LABEL']

Obtenemos la matriz de correlación de las variables independientes con la idea de eliminar aquellas variables muy correlacionadas entre si.  Dado que la matriz es simétrica, se deja solo la mitad inferior

In [103]:
# calculamos correlación de variables independientes
corr_matrix = X.corr()

import numpy as np
mitad_inferior = np.tril(corr_matrix, k=-1)
mitad_inferior = pd.DataFrame(mitad_inferior, columns=corr_matrix.columns, index=corr_matrix.index)

Se realiza un estudio por umbral, para determinar cuantas variables serían eliminadas y cuantas quedarían.   Entre más alto el umbral, las variables independientes están menos correlacionadas entre si

In [106]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score

lista_umbral = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

for umbral in lista_umbral:
    j = 0
    variables_eliminar = set()
    for c in mitad_inferior.columns:
        i = 0
        for v in mitad_inferior[c]:
            if v > umbral:
                variables_eliminar.add(mitad_inferior.index[i])           
            i = i + 1
        j = j + 1
    print('umbral:', 
          umbral, len(mitad_inferior.columns), 
          len(variables_eliminar),
          len(mitad_inferior.columns) - len(variables_eliminar))
    
    Xnew = X.drop(list(variables_eliminar), axis=1)

    # Separar los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(Xnew, y, test_size=0.2, random_state=42)

    print(y_test.value_counts())

    # Aplicar StandardScaler para estandarizar las características (opcional pero recomendado)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Entrenar el modelo de regresión logística
    model = LogisticRegression(max_iter=3000)
    cv_scores = cross_val_score(model, X_train, y_train, cv=5)  # Utiliza 5 divisiones para validación cruzada

    # Imprimir los puntajes de validación cruzada
    print("Puntajes de validación cruzada:", cv_scores)
    print("Precisión media: {:.2f}".format(cv_scores.mean()))

    model.fit(X_train, y_train)

    y_pred_prob = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)
    roc_auc = roc_auc_score(y_test, y_pred_prob)
    accuracy_test = accuracy_score(y_test, y_pred)

    print('roc-auc:', roc_auc, 'accuracy:', accuracy_test)



            


umbral: 0.9 1953 490 1463
LABEL
0    970
1    353
Name: count, dtype: int64
Puntajes de validación cruzada: [0.80340265 0.83364839 0.80812854 0.83349101 0.83254494]
Precisión media: 0.82
roc-auc: 0.8864285505680325 accuracy: 0.8480725623582767
umbral: 0.8 1953 584 1369
LABEL
0    970
1    353
Name: count, dtype: int64
Puntajes de validación cruzada: [0.79773157 0.83648393 0.80718336 0.83538316 0.82403027]
Precisión media: 0.82
roc-auc: 0.8842878420606877 accuracy: 0.8420256991685563
umbral: 0.7 1953 703 1250
LABEL
0    970
1    353
Name: count, dtype: int64
Puntajes de validación cruzada: [0.79678639 0.83931947 0.80056711 0.83632923 0.81078524]
Precisión media: 0.82
roc-auc: 0.8770333810344324 accuracy: 0.8412698412698413
umbral: 0.6 1953 799 1154
LABEL
0    970
1    353
Name: count, dtype: int64
Puntajes de validación cruzada: [0.79678639 0.81663516 0.79111531 0.83159886 0.8051088 ]
Precisión media: 0.81
roc-auc: 0.8713034081948542 accuracy: 0.8269085411942555
umbral: 0.5 1953 915 103